In [ ]:
import plotly.io as pio
pio.renderers.default = "browser"

In [2]:
import plotly.graph_objects as go
import plotly.io as pio
import pandas as pd
import utils
import instrument

In [3]:
pair = ''
granularity = ''
ma_list = [8, 12]
i_pair = ''

In [4]:
df = pd.read_csv('BTC_USD_1m_Nov.csv')
df_plot = df.iloc[0:2500].copy()

In [5]:

#df_ma = df_plot[['open_time','close','volume']].copy()
df_ma = df_plot[['open_time','close']].copy()

In [6]:
for ma in ma_list:
    df_ma[f'MA_{ma}'] = df_ma.close.rolling(window=ma).mean()
df_ma.dropna(inplace=True)
df_ma.reset_index(drop=True, inplace=True)


In [7]:
df_ma['DIFF'] = df_ma.MA_8 - df_ma.MA_12 #calculate a difference between two moving averages when "creating a cross"
df_ma['DIFF_PREV'] = df_ma.DIFF.shift(1) #gives previous row value


In [11]:
def is_trade(row): #function to identify crosses/trades
    if row.DIFF >= 0 and row.DIFF_PREV < 0:
        return 1 # Golden Cross
    if row.DIFF <= 0 and row.DIFF_PREV > 0:
        return -1 # Death Cross
    return 0

In [ ]:
#Make new column, apply 
df_ma['IS_TRADE'] = df_ma.apply(is_trade, axis=1) #axis means apply fuction to each row
#create dataframe with made trades
df_trades = df_ma[df_ma.IS_TRADE != 0].copy()
df_ma.reset_index(drop=True, inplace=True)
df_trades.shape
df_trades.head()

,open_time,close,MA_8,MA_12,DIFF,DIFF_PREV,IS_TRADE
3,2025-11-01 00:14:00+00:00,109471.44,109454.25250,109449.895833,4.356667,-0.369583,True
14,2025-11-01 00:25:00+00:00,109519.99,109479.51000,109479.613333,-0.103333,3.364167,True
16,2025-11-01 00:27:00+00:00,109531.94,109492.92375,109492.157500,0.766250,-1.109167,True
23,2025-11-01 00:34:00+00:00,109445.34,109480.96750,109491.249167,-10.281667,4.573333,True
30,2025-11-01 00:41:00+00:00,109492.00,109481.82125,109472.561667,9.259583,-1.875417,True


In [10]:
fig = go.Figure()

fig.add_trace(go.Candlestick(   #Adding candlestick chart
    x=df_plot['open_time'], #candlesticks
    open=df_plot['open'],
    high=df_plot['high'],
    low=df_plot['low'],
    close=df_plot['close'],
    #volume=df_plot['volume'],
    line=dict(width=1), opacity=1, #coloring
    increasing_line_color='#2EC866', 
    decreasing_line_color='#FF3A4C',
    increasing_fillcolor='#24A06B', 
    decreasing_fillcolor='#CC2E3B',
    )) 

for ma in ma_list:
    col = f'MA_{ma}'
    fig.add_trace(go.Scatter(    #Adding Moving Average line
        x=df_plot.open_time,
        y=df_ma[col],
        line=dict(width=2),
        line_shape='spline', #makes smooth curve
        name=col
        ))

fig.update_layout(width=1100, height=450,
                  margin=dict(l=30, r=30, t=30, b=30),
                  font=dict(color='#e1e1e1'), 
                  paper_bgcolor='#1e1e1e',
                  plot_bgcolor='#1e1e1e')

fig.update_xaxes(
    gridcolor='#1f292f',
    showgrid=True,fixedrange=True,rangeslider=dict(visible=False)
)
 
fig.update_yaxes(
    gridcolor='#1f292f',
    showgrid=True
)
fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed